In [ ]:
!pip install fiftyone hf-transfer ultralytics

In [ ]:
!pip install -U -q git+https://github.com/huggingface/transformers

In [ ]:
import os

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
from ultralytics import YOLO

yolo_world_model = YOLO("yolov8x-worldv2.pt")  # or choose yolov8m/l-world.pt

# Define custom classes
yolo_world_model.set_classes(["person"])


In [ ]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh


dataset = fouh.load_from_hub(
    "Voxel51/MPII_Human_Pose_Dataset",
    name="human_pose",
    max_samples = 10,
    overwrite=True,
    shuffle=True
    )

In [ ]:
dataset.compute_metadata()

In [ ]:
dataset.apply_model(
    yolo_world_model,
    label_field="bbox_detections"
)

In [ ]:
!fiftyone plugins download https://github.com/harpreetsahota204/vitpose-plugin

In [ ]:
import os

os.environ['FIFTYONE_ALLOW_LEGACY_ORCHESTRATORS'] = 'true'

In [ ]:
fo.launch_app(dataset)

To use via SDK:

In [ ]:
import fiftyone.operators as foo

vitpose_operator = foo.get_operator("@harpreetsahota/vitpose/vitpose_keypoint_estimator")

If you're running in a notebook, it's recommended to launch a Delegated operation by running `fiftyone delegated launch` in terminal, then run as follows:

In [ ]:
await vitpose_operator(
    dataset,
    model_name="usyd-community/vitpose-plus-small",
    bbox_field="bbox_field",
    output_field="vitpose_estimates",
    confidence_threshold= 0.55,
    delegate=True
)

And you can see the result added to your dataset:


In [ ]:
dataset